Importiere die requests Library

In [4]:
import requests

Importiere die Pandas Library in der Kurzform.

In [209]:
import pandas as pd
import time 

1. Unter Verwendung der [USGS.gov Earthquake API](https://earthquake.usgs.gov) ziehe heraus, wieviele Erdbeben sich im August 2018 ereignet haben?

In [15]:
url = 'https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2018-08-01&endtime=2018-08-31'

In [16]:
response = requests.get(url)

In [17]:
response.text

'17045'

2. Mache das gleiche für August 2017, 2016, 2015?

In [18]:
url = 'https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2017-08-01&endtime=2017-08-31'

In [19]:
response = requests.get(url)

In [20]:
response.text

'12300'

In [21]:
url = 'https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2016-08-01&endtime=2016-08-31'

In [22]:
response = requests.get(url)

In [23]:
response.text

'10878'

In [93]:
url = 'https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2015-08-01&endtime=2015-08-31'

In [94]:
response = requests.get(url)

In [96]:
response.text

'9981'

In [115]:
# Oder eine noch schickere Variante!

# https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2014-01-01&endtime=2014-01-02

import requests
import pandas as pd
base = "https://earthquake.usgs.gov/fdsnws/event/1/"
action = "count?" # Der count? kann man auch easy durch query ersetzen.

# Url = "starttime=2018-09-25&endtime=2018-09-30-25&endtime=2018-09-30"

erdbebenzahl = []

for jahr in range(2015, 2019): # Kleiner Loop, um die Range zu bestimmen
    
    url = base + action + "starttime=" + str(jahr) + "-08-01&endtime=" + str(jahr) + "-08-31"
    
    response = requests.get(url)
    print(str(jahr) + ":" + str(response.text))
    
    minidict = {'Jahr': jahr,
               'Anzahl': str(response.text)}
    
    erdbebenzahl.append(minidict)

2015:9981
2016:10878
2017:12300
2018:17045


In [112]:
pd.DataFrame(erdbebenzahl)

,Anzahl,Jahr
0,9981,2015
1,10878,2016
2,12300,2017
3,17045,2018


In [122]:
#meine Lösung. 
liste_beben=[]
for year in range(2015,2019):
    r= requests.get("https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=" +str(year)+"-08-01&endtime="+ str(year)+"-08-31")
  
    liste_beben.append(r.text)
print(liste_beben)
    
#print umbedingt wie hier gemacht wieder aus dem Loop nehmen, sonst druckt er jedesmal aus.


['9981', '10878', '12300', '17045']


In [123]:
pd.DataFrame(liste_beben)

,0
0,9981
1,10878
2,12300
3,17045


3. Zeige nur die Erdbeben an, die in den letzten 5 Jahren eine Mindeststärke von 5,5 hatten? (Neue Parameter kannst Du mit den Zeichen "&" ergänzen. Lese Dich in die Dokumentation ein, um noch mehr Parameter kennezulernen)

In [174]:
erdbeben_55 = requests.get('https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2014-09-01&endtime=2019-09-23&minmagnitude=5.5')
erdbeben_55.text

'2255'

4. Zeige eine Liste der 10 stärksten Erdbeben des laufenden Jahres an. (Nicht vergessen, nun musst Du die Methode wechseln. Nicht mehr "count", sondern "query").

In [200]:
erdbeben = requests.get("https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2019-01-01&endtime=2019-09-24&minmagnitude=5.5")
erdbeben = erdbeben.json()
erdbeben.keys()
erdbebenliste = erdbeben['features']
leereliste = []
for erdbeben in erdbebenliste:
    mag=erdbeben["properties"]["mag"]
    #print (mag)
    try:
        if mag >= 5.5:
            leereliste.append(mag)
    except:
        if None:
            continue

In [201]:
leereliste.sort()

In [202]:
len(leereliste)

323

In [205]:
leereliste[-10:]

[6.9, 7, 7.1, 7.1, 7.2, 7.3, 7.3, 7.5, 7.6, 8]

5. Wo haben sich diese Erdbeben ereignet?

In [227]:
# Wir kennen nun wo sich die 10 stärksten (mag), wann (time) und an welchem Ort (place) ereignet haben. 

from datetime import datetime

erdbeben_max = requests.get('https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2019-01-01&endtime=2019-09-24&minmagnitude=6.5')
erdbeben_max = erdbeben_max.json() # json macht ein liste von mini-dictionarys - sonst unleserlich.
erdbeben_max = erdbeben_max["features"] # In den Features sind die Daten drin, die uns interessieren.

top2019 = [] # Leere Liste kreieren.

for beben in erdbeben_max: # Loop: suche nach mag, place, time.
    magnitude = beben["properties"]["mag"]
    place = beben["properties"]["place"]
    
    # So definieren wir die time im Loop.
    zeit = str(beben["properties"]["time"])[:-3] #[:-3] --> Die letzten drei Ziffern bei der Unixtime müssen weg.
    zeit = datetime.utcfromtimestamp(int(zeit)) # Hier wird die Zeitumgewandelt in Time. 
    zeit = zeit.strftime('%Y-%m-%d %H:%M:%S') # Auf Liste nachgucken!
    
    mini_dict = {"Magnitude: ":magnitude, "Place: ":place, "Time: ":zeit} # Dict. definieren mit Keys und Values.
    
    top2019.append(mini_dict) # Leere Liste in einen Dict füllen.
    
pd.DataFrame(top2019).sort_values("Magnitude: ", ascending=False)[:10] #[:10] --> Sind sind die zehn stärksten Erdbeben.

,Magnitude:,Place:,Time:
12,8.0,"78km SE of Lagunas, Peru",2019-05-26 07:41:15
13,7.6,"46km SSE of Namatanai, Papua New Guinea",2019-05-14 12:58:25
18,7.5,"111km ESE of Palora, Ecuador",2019-02-22 10:17:23
9,7.3,"292km NW of Saumlaki, Indonesia",2019-06-24 02:53:39
10,7.3,"116km NE of L'Esperance Rock, New Zealand",2019-06-15 22:55:04
5,7.2,"93km NNE of Laiwui, Indonesia",2019-07-14 09:10:51
8,7.1,2019 Ridgecrest Earthquake Sequence,2019-07-06 03:19:53
14,7.1,"33km NW of Bulolo, Papua New Guinea",2019-05-06 21:19:37
17,7.0,"23km NNE of Azangaro, Peru",2019-03-01 08:50:42
2,6.9,"106km WSW of Tugu Hilir, Indonesia",2019-08-02 12:03:26


In [230]:
print(top2019.append(mini_dict))

None
